In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from sophia import SophiaG

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# show images
#imshow(torchvision.utils.make_grid(images))
# print labels
#print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

In [4]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        criterion = nn.CrossEntropyLoss()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x, y):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        if y is not None:
            loss = criterion(x, y)
        else:
            loss = None
        return x, loss


model = Net()

In [5]:
import torch.optim as optim


optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [6]:
for epoch in range(4):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs, loss = model(inputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.212
[1,  4000] loss: 1.860
[1,  6000] loss: 1.680
[1,  8000] loss: 1.590
[1, 10000] loss: 1.499
[1, 12000] loss: 1.444
[2,  2000] loss: 1.374
[2,  4000] loss: 1.369
[2,  6000] loss: 1.349
[2,  8000] loss: 1.313
[2, 10000] loss: 1.274
[2, 12000] loss: 1.266
[3,  2000] loss: 1.212
[3,  4000] loss: 1.198
[3,  6000] loss: 1.189
[3,  8000] loss: 1.187
[3, 10000] loss: 1.160
[3, 12000] loss: 1.183
[4,  2000] loss: 1.098
[4,  4000] loss: 1.081
[4,  6000] loss: 1.113
[4,  8000] loss: 1.104
[4, 10000] loss: 1.075
[4, 12000] loss: 1.105
Finished Training


In [7]:
model = Net()

In [8]:
optimizer = SophiaG(model.parameters(), lr=1e-4, betas=(0.965, 0.99), rho=0.05, weight_decay=1e-1)

total_bs = len(trainloader)
bs = batch_size
k = 10
iter_num = -1
epochs = 10
# training loop
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader):
        # standard training code
        X, Y = data
        logits, loss = model(X, Y)
        loss.backward()
        optimizer.step(bs=bs)
        optimizer.zero_grad(set_to_none=True)
        iter_num += 1
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

        if iter_num % k != k - 1:
            continue
        else:
            # update hessian EMA
            logits, _ = model(X, None)
            samp_dist = torch.distributions.Categorical(logits=logits)
            y_sample = samp_dist.sample()
            loss_sampled = F.cross_entropy(logits.view(-1, logits.size(-1)), y_sample.view(-1), ignore_index=-1)
            loss_sampled.backward()
            optimizer.update_hessian()
            optimizer.zero_grad(set_to_none=True)
            model.zero_grad()

[1,  2000] loss: 1.950
[1,  4000] loss: 1.691
[1,  6000] loss: 1.605
[1,  8000] loss: 1.521
[1, 10000] loss: 1.488
[1, 12000] loss: 1.423
[2,  2000] loss: 1.374
[2,  4000] loss: 1.342
[2,  6000] loss: 1.324
[2,  8000] loss: 1.273
[2, 10000] loss: 1.279
[2, 12000] loss: 1.265
[3,  2000] loss: 1.199
[3,  4000] loss: 1.206
[3,  6000] loss: 1.186
[3,  8000] loss: 1.207
[3, 10000] loss: 1.185
[3, 12000] loss: 1.173
[4,  2000] loss: 1.143
[4,  4000] loss: 1.123
[4,  6000] loss: 1.119
[4,  8000] loss: 1.121
[4, 10000] loss: 1.136
[4, 12000] loss: 1.113
[5,  2000] loss: 1.072
[5,  4000] loss: 1.080
[5,  6000] loss: 1.063
[5,  8000] loss: 1.059
[5, 10000] loss: 1.099
[5, 12000] loss: 1.119
[6,  2000] loss: 1.042
[6,  4000] loss: 1.052
[6,  6000] loss: 1.055
[6,  8000] loss: 1.044
[6, 10000] loss: 1.059
[6, 12000] loss: 1.048
[7,  2000] loss: 1.010
[7,  4000] loss: 1.003
[7,  6000] loss: 1.039
[7,  8000] loss: 1.038
[7, 10000] loss: 1.006
[7, 12000] loss: 1.032
[8,  2000] loss: 0.992
[8,  4000] 